<a href="https://colab.research.google.com/github/aCoderChild/Kaggle-and-AI/blob/main/final%20test/XGBOOST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
import math
import numpy as np
import pandas as pd
from collections import defaultdict
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/aCoderChild/Kaggle-and-AI/refs/heads/main/final%20test/data/train.csv')
df_test = pd.read_csv('https://raw.githubusercontent.com/aCoderChild/Kaggle-and-AI/refs/heads/main/final%20test/data/test.csv')

In [4]:
def preprocessing_data(df):
    df['person_education'].replace({
            'High School': 0,
            'Associate': 1,
            'Bachelor': 2,
            'Master': 3,
            'Doctorate':4
        }, inplace=True)

    gender_mapping = {'male': 0, 'female': 1}
    home_ownership_mapping = {'RENT': 0, 'OWN': 1, 'MORTGAGE': 2, 'OTHER': 3}
    loan_intent_mapping = {'PERSONAL': 0, 'EDUCATION': 1, 'MEDICAL': 2, 'VENTURE': 3, 'HOMEIMPROVEMENT': 4, 'DEBTCONSOLIDATION': 5}
    previous_loan_defaults_mapping = {'No': 0, 'Yes': 1}


    df['person_gender'] = df['person_gender'].map(gender_mapping)
    df['person_home_ownership'] = df['person_home_ownership'].map(home_ownership_mapping)
    df['loan_intent'] = df['loan_intent'].map(loan_intent_mapping)
    df['previous_loan_defaults_on_file'] = df['previous_loan_defaults_on_file'].map(previous_loan_defaults_mapping)

    return df # preprocess the data, turning the String data into int for easier training

In [5]:
df = preprocessing_data(df)
df_test = preprocessing_data(df_test) # preprocess the training tand testing data

<ipython-input-4-4c67094b7d09>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['person_education'].replace({
<ipython-input-4-4c67094b7d09>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['person_education'].replace({
<ipython-input-4-4c67094b7d09>:2: FutureWarning: A value is trying to be s

In [6]:
df # print the data

,person_age,person_gender,person_education,person_income,person_emp_exp,person_home_ownership,loan_amnt,loan_intent,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,previous_loan_defaults_on_file,loan_status
0,22.0,1,3,71948.0,0,0,35000.0,0,16.02,0.49,3.0,561,0,1
1,21.0,1,0,12282.0,0,1,1000.0,1,11.14,0.08,2.0,504,1,0
2,25.0,1,0,12438.0,3,2,5500.0,2,12.87,0.44,3.0,635,0,1
3,23.0,1,2,79753.0,0,0,35000.0,2,15.23,0.44,2.0,675,0,1
4,24.0,0,3,66135.0,1,0,35000.0,2,14.27,0.53,4.0,586,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32994,35.0,0,0,87428.0,10,2,12888.0,2,6.47,0.15,7.0,664,1,0
32995,26.0,1,2,91318.0,6,2,8000.0,3,14.92,0.09,5.0,590,1,0
32996,23.0,1,2,79749.0,0,0,16800.0,0,11.28,0.21,2.0,632,0,0
32997,25.0,0,0,51450.0,1,0,6000.0,2,12.48,0.12,3.0,661,0,0


In [7]:
X = df.drop(['loan_status'], axis = 1)
y = df['loan_status']
# X and Y vectors of training data

In [8]:
XX = df_test.drop(['loan_status'], axis = 1)
yy = df_test['loan_status']
# X and Y vectors of testing data

# XGBoost Implementation

In [9]:
class XGBClassifier():
    '''
        Model represents part of XGBoost model in Python library
        Based on the paper about XGBoost: https://www.kdd.org/kdd2016/papers/files/rfp0697-chenAemb.pdf
        The Model does NOT represent full features of xgboostclassifier in xgb library
        Model uses tree_method = 'exact'
    '''
    def __init__(self, params, random_seed=None):
        '''
            Constructor XGBoostModel

            params: A dictionary of hyperparameters used for training the model
            (like learning rate, maximum depth, etc.).
            If any parameter is missing, it will use a default value.

            random_seed: random numbers - seed for randoms
        '''
        self.params = defaultdict(lambda: None, params)

        # Subsample: fraction of data to use for each boosting round. If not specified, it uses 1.0 (i.e., the entire dataset)
        if 'subsample' in self.params and self.params['subsample']:
            self.subsample = self.params['subsample']
        else:
            self.subsample = 1.0

        # Learning rate in [0,1]: Controls how much each tree should contribute to the final prediction. It defaults to 0.3
        if 'learning_rate' in self.params and self.params['learning_rate']:
            self.learning_rate = self.params['learning_rate']
        else:
            self.learning_rate = 0.3

        # Base predict score: The initial prediction for all samples. If not specified, it defaults to 0.5.
        if 'base_score' in params and params['base_score']:
            self.base_score = params['base_score']
        else:
            self.base_score = 0.5

        # Maximum depth of a decision tree, value in [1,∞]. If not specified, it defaults to 5
        if 'max_depth' in params and params['max_depth']:
            self.max_depth = params['max_depth']
        else:
            self.max_depth = 5

        # Random_seed
        self.rng = np.random.default_rng(seed=random_seed)


    def fit(self, X, y, LossFunction, n_estimators):
        '''
            Provide data for the model and execute training

            X: training data
            y: training label
            LossFunction: target function for training
            n_estimators: a number of boosting iterations or the number of decision trees built
        '''
        # Starts with a baseline prediction (base_score) for all samples, which is initially set to 0.5. This will be updated as trees are added.
        current_predictions = self.base_score * np.ones(shape=y.shape)

        # TreeBoosters: A list to store the individual decision trees (boosters) built during training.
        self.TreeBoosters = []
        for i in range(n_estimators):

            # Calculate gradients and hessians based on previous predictions
            # First and second derivatives of the loss function
            # They represent how much the current model's predictions need to change to improve
            # Computed based on the currnet_predictions and the actual target values
            gradients = LossFunction.gradient(y, current_predictions)
            hessians = LossFunction.hessian(y, current_predictions)

            # Data Subsampling:
            #    If subsample is less than 1, a random subset (reduce overfitting) of the training data is selected for the current round.
            #    If subsample equals 1, the entire dataset is used.
            if self.subsample != 1.0:
                sample_size = math.floor(self.subsample * len(y))
                sample_idxs = self.rng.choice(len(y), size=sample_size, replace=False)
            else:
                sample_idxs = None

            # Tree Building (Booster): A new tree (booster) is constructed using the gradients and hessians.
            # It uses the training data, gradients, and hessians to build a decision tree.
            booster = TreeBooster(X, gradients, hessians, self.params, self.max_depth, sample_idxs)

            # Update Predictions: The current model’s predictions are updated by adding the new tree’s predictions, scaled by the learning_rate.
            current_predictions += self.learning_rate * booster.predict(X)

            # Store the Booster: The newly created tree is stored in the TreeBoosters list for later use in prediction.
            self.TreeBoosters.append(booster)


    def predict(self, X):
        '''
            predict the result based on the input data
        '''
        # Tree Contributions: For each tree in TreeBoosters, the predict method of each tree is called to get its contribution to the prediction for each sample in X.
        TreeBoosters_predictions = [booster.predict(X) for booster in self.TreeBoosters]
        # Calculate the total prediction
        total_TreeBoosters_contribution = np.sum(TreeBoosters_predictions, axis=0)
        # Calculate the predict
        prediction = self.base_score + self.learning_rate * total_TreeBoosters_contribution

        # Do predict for binary classification problem
        y_pred = []
        for i in prediction:
            if i < 0.5:
                y_pred.append(0)
            else:
                y_pred.append(1)
        y_pred = np.array(y_pred)

        return y_pred

In [10]:
'''
    The TreeBooster class is responsible for building and handling individual decision trees during the boosting process.

    These trees are built iteratively and are designed to correct the residual errors of previous trees in the boosting sequence.

    The key functions of this class are:
       - Initialization (__init__): This sets up the tree and calculates the value at the root node.
       - Child Node Creation (_maybe_insert_child_nodes): This tries to split the data further by finding the best split for each feature.
       - Finding the Best Split (_find_better_split): This finds the best threshold for splitting the data based on a specific feature, considering both gradients and hessians.
       - Prediction (predict): This makes predictions by traversing the tree.
       - Leaf Node Check (is_leaf): This checks if the current node is a leaf node, meaning no further splits are needed.
'''

class TreeBooster():

    def __init__(self, X, g, h, params, max_depth, idxs=None):
        '''
            Costruct Decision Tree or Subtree of Decision Tree (based on max_depth)

            X: feature matrix (training data)
            g: the gradients (first derivatives of loss function)
            h: the hessians (second derivatives of loss function)
            params: Hyperparameters such as regulatization, learning rate, etc.
            max_depth: maximum depth of a tree, which controls how many levels of splitting the tree will have
            idxs: The indices of the data points used in the current split. If None, it means the whole dataset is used.
        '''

        self.params = params
        self.max_depth = max_depth

        # Regularization hyperparameters
        # They help prevent overfitting and control the model's complexity by adding constraints to how the tree is built

        # Controls the minimum amount of "cover" (or sum of hessians) required in a child node.
        # It helps in preventing overfitting by avoiding splits that result in overly small nodes with insufficient data.
        self.min_child_weight = params.get('min_child_weight', 1.0)

        # reg_lambda: L2 regularization
        # This parameter adds a regularization term to the objective function, helping to control the complexity of the model by penalizing large weights.
        self.reg_lambda = params.get('reg_lambda', 1.0)

        # Specifies the minimum loss reduction required to make a further split.
        # It sets a threshold for how much the gain (improvement in the objective function) must be for a split to be accepted.
        # If the gain from a split is less than gamma, the split is rejected, and the tree doesn't grow further at that node.
        self.gamma = params.get('gamma', 0.0)

        # Converts the gradients (g) and hessians (h) to NumPy arrays for easier calculations.
        if isinstance(g, pd.Series):
            g = g.values
        if isinstance(h, pd.Series):
            h = h.values

        # idxs = None <=> whole dataset
        if idxs is None:
            idxs = np.arange(len(g))

        self.X = X  # Training data

        self.g = g  # Calculated gradients during previous iterations
        self.h = h  # Calculated Hessians during previous iterations

        self.idxs = idxs    # Index of the data of X will be used
        self.n = len(idxs)  # Number of samples
        self.c = X.shape[1]  # Number of features

        # Computes the initial value for the root node
        self.value = -g[idxs].sum() / (h[idxs].sum() + self.reg_lambda)
        # This is the predicted value for the node before any splitting.

        # track the best split found during the process
        self.best_score_so_far = 0.

        # Recursion to split:
        # If the tree's depth (max_depth) is greater than 0,
        # the _maybe_insert_child_nodes method is called to attempt splitting the node further.
        if self.max_depth > 0:
            self._maybe_insert_child_nodes()


    def _maybe_insert_child_nodes(self):

        # It loops through all features (columns of X) and tries to find the best possible split
        # using the _find_better_split method.
        for feature_idx in range(self.c):
            self._find_better_split(feature_idx)

        # If it is leaf node, stop
        if self.is_leaf:
            return

        # Take the value of the chosen feature for splitting
        x = self.X.values[self.idxs, self.split_feature_idx]

        # Divide the data into 2 groups based on the threshold calculated in _find_better_split()
        left_idx = np.nonzero(x <= self.threshold)[0]
        right_idx = np.nonzero(x > self.threshold)[0]

        # Construct Left Node
        self.left = TreeBooster(
            self.X,
            self.g,
            self.h,
            self.params,
            self.max_depth - 1, # -1 max_depth
            self.idxs[left_idx]
        )

        # Construct Right Node
        self.right = TreeBooster(
            self.X,
            self.g,
            self.h,
            self.params,
            self.max_depth - 1, # -1 max_depth
            self.idxs[right_idx]
        )


    @property
    def is_leaf(self):
        '''
            Check if the current Node is Leaf Node or not
            If the splitting does NOT increase Gain => This Node is a Leaf Node
        '''
        if self.best_score_so_far == 0:
            return True
        return False


    def _find_better_split(self, feature_idx):
        '''
            Function finds the optimal splitting for Decision Tree
            The Searching is practised by choosing 2 consecutive value in a sorted array
            Taking the average of the 2 numbers as threshold. After that calculate Gain according to the threshold.
            Finally, choose the threshold with the highest Gain

            feature_idx: feature (column) - defined
        '''

        # Retreive feature value and gradient/hessian in respect
        x = self.X.values[self.idxs, feature_idx]
        g = self.g[self.idxs]
        h = self.h[self.idxs]

        # For the given feature (feature_idx), the values of that feature are sorted,
        # along with the corresponding gradients (g) and hessians (h).
        sort_idx = np.argsort(x)
        sort_x = x[sort_idx]
        sort_g = g[sort_idx]
        sort_h = h[sort_idx]

        # Total gradient (total_g) and total hessian (total_h) for the entire dataset.
        total_g = g.sum()
        total_h = h.sum()
        sum_g_left, sum_h_left = 0.0, 0.0
        sum_g_right, sum_h_right = total_g, total_h

        # It then iterates through the sorted values and calculates the "gain" of splitting the data at each possible threshold.
        '''
            'exact': threshold is the average of two consecutive feature values
        '''
        for i in range(self.n - 1):
            g_i = sort_g[i]
            h_i = sort_h[i]
            x_i = sort_x[i]
            x_i_next = sort_x[i + 1] # Next value

            # Update gradients và hessians of Left and Right Nodes
            sum_g_left += g_i
            sum_h_left += h_i
            sum_g_right -= g_i
            sum_h_right -= h_i

            # Check Splitting condition
            # If Cover smaller than min Cover: reject the node and stop splitting
            # Cover = sigma h_i
            if sum_h_left < self.min_child_weight or x_i == x_i_next:
                continue
            if sum_h_right < self.min_child_weight:
                break

            # Calculate Gain of the splitting according to formula (7) in paper
            gain = (
                0.5
                * (
                    (sum_g_left**2 / (sum_h_left + self.reg_lambda))
                    + (sum_g_right**2 / (sum_h_right + self.reg_lambda))
                    - (total_g**2 / (total_h + self.reg_lambda))
                )
                - self.gamma / 2
            )

            # Update if greater Gain found
            if gain > self.best_score_so_far:
                self.split_feature_idx = feature_idx
                self.best_score_so_far = gain
                self.threshold = (x_i + x_i_next) / 2


    def predict(self, X):
        """
        This method is responsible for making predictions using the trained tree.

        It iterates over each row of the input X (which is a DataFrame), and for each row,
        it calls _predict_row to traverse the tree and get the prediction for that specific row.
        """
        predictions = []
        for _, row in X.iterrows():
            predictions.append(self._predict_row(row))
        return np.array(predictions)


    def _predict_row(self, row):
        """
        Predict the value for a row of data
        """
        # If the current node is a leaf, it returns the value stored at the leaf node (the predicted value for that node).
        if self.is_leaf:
            return self.value

        # If the current node is not a leaf, it checks which child node (left or right)
        # the row should go to based on the value of the feature at split_feature_idx and the threshold (threshold).
        if row.iloc[self.split_feature_idx] <= self.threshold:
            child = self.left
        else:
            child = self.right

        # The method recursively calls _predict_row on the appropriate child node until it reaches a leaf node,
        # and then it returns the predicted value at that leaf.
        return child._predict_row(row)

In [11]:
class SquaredErrorObjective:
    def loss(self, y, pred):
        """
        Calculate Mean Squared Error (MSE).
        """
        mse = np.mean((y - pred) ** 2)
        return mse

    def gradient(self, y, pred):
        """
        Calculate Gradient of Mean Squared Error (MSE) in respect to each predicted value
        First derivatives of loss function
        """
        grad = pred - y
        return grad

    def hessian(self, y, pred):
        """
        Calculate hessian of MSE.
        Second derivatives of loss function
        """
        hess = np.ones(len(y))
        return hess

In [12]:
'''
The class LogLossObjective defines a custom loss function used in binary classification tasks,
specifically for logistic regression or binary classification problems where the output is a probability.
'''
class LogLossObjective:
    def loss(self, y, pred):
        """
        This method computes the log loss (also known as binary cross-entropy)
        between the true labels (y) and the predicted probabilities (pred).
        """
        # epsilon = 1e-15  # Avoid log(0)
        # pred = np.clip(pred, epsilon, 1 - epsilon)  # Limit the value to ensure stability
        log_loss = -(y * np.log(pred) + (1 - y) * np.log(1 - pred))
        # pred: The predicted probabilities, outputs from a model that has passed through a sigmoid function, ranging from 0 to 1.
        return log_loss

    def gradient(self, y, pred):
        """
        This method calculates the gradient (first derivative) of the log loss function
        with respect to the predicted values (pred).
        """
        pred = 1.0 / (1.0 + np.exp(-pred))
        grad = pred - y
        return grad

    def hessian(self, y, pred):
        """
        This method calculates the Hessian (second derivative) of the log loss function
        with respect to the predicted values (pred).

        This is the second derivative of the log loss function and measures the curvature or the rate of change of the gradient.
        """
        pred = 1.0 / (1.0 + np.exp(-pred))
        hess = pred*(1-pred)
        return hess

Train model

In [ ]:
# multiple
import time
import random

for i in range(0,100):
    print(f"Times: {i+1}")
    start_time = time.time()

    params = {
        'learning_rate': random.uniform(0.2, 0.4),
        'max_depth': random.randint(12, 20),
        'subsample': random.uniform(0.7, 0.9),
        'reg_lambda': 1.0,
        'gamma': 0.0,
        'min_child_weight': random.uniform(1.0, 3.0),
        'base_score': 0.75,
    }
    n_estimators = random.randint(50, 100)

    # train the from-scratch XGBoost model
    model_scratch = XGBClassifier(params, random_seed=42)
    # Can use 1 of 2 loss functions:
    # LogLossObjective() or SquaredErrorObjective()
    model_scratch.fit(X, y, LogLossObjective(), n_estimators) #fit model

    end_time = time.time()

    # Running time
    elapsed_train_time = end_time - start_time

    start_time = time.time()

    pred_scratch = model_scratch.predict(XX)

    end_time = time.time()
    elapsed_test_time = end_time - start_time

    accuracy = accuracy_score(yy, pred_scratch)
    f1 = f1_score(yy, pred_scratch)
    precision = precision_score(yy, pred_scratch)
    recall = recall_score(yy, pred_scratch)
    # confusion_matrix = confusion_matrix(yy, pred_scratch)
    print(f"Accuracy: {accuracy * 100:.2f}%")
    print(f"F1 Score: {f1 * 100:.2f}%")
    print(f"Precision: {precision * 100:.2f}%")
    print(f"Recall: {recall * 100:.2f}%")
    print("")
    print(params)
    print(f"n_estimators: {n_estimators}")

    print(f"Train running time: {elapsed_train_time:.6f} giây")
    print(f"Test: {elapsed_test_time:.6f} giây")
    print("")
    print("")

Times: 1
Accuracy: 93.42%
F1 Score: 85.35%
Precision: 92.14%
Recall: 79.50%

{'learning_rate': 0.37708681042741465, 'max_depth': 14, 'subsample': 0.8276106690828566, 'reg_lambda': 1.0, 'gamma': 0.0, 'min_child_weight': 2.327996464162829, 'base_score': 0.75}
n_estimators: 97
Train running time: 956.773480 giây
Test: 125.543827 giây


Times: 2
Accuracy: 93.55%
F1 Score: 85.61%
Precision: 92.60%
Recall: 79.60%

{'learning_rate': 0.28350321158658337, 'max_depth': 20, 'subsample': 0.8491392043936676, 'reg_lambda': 1.0, 'gamma': 0.0, 'min_child_weight': 1.6515391642057178, 'base_score': 0.75}
n_estimators: 98
Train running time: 1156.364107 giây
Test: 133.794685 giây


Times: 3
Accuracy: 92.98%
F1 Score: 84.31%
Precision: 91.43%
Recall: 78.22%

{'learning_rate': 0.3776527486703133, 'max_depth': 17, 'subsample': 0.7731868148299113, 'reg_lambda': 1.0, 'gamma': 0.0, 'min_child_weight': 1.5739570998408616, 'base_score': 0.75}
n_estimators: 66
Train running time: 706.292588 giây
Test: 85.290113 g